In [1]:
from ovcg import *
from pyccx import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor, AbsCovergenceMonitor
from mapper import generate_mapper

In [2]:
ccx_logoff = True
if ccx_logoff:
    import logging
    logger = logging.getLogger('PYCCX')
    logger.setLevel(logging.CRITICAL)

use_relax = True

# tolerance setting
tol = 1e-6
if tol > 1e-1:
    print('The tolerance is too large, this may lead to instability or low quality results')

In [3]:
# fluid solver
interface_grid = 2
interface_side = 0
interface_axis = 1

solverF = Cgins()
solverF.read_cmd('fluid/fluidD.cmd')
solverF.init_grid()
solverF.init_temperature_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid/dummyD')
fluid_ofile = VtkFile('fluid/resFDSN_ori.pvd')
fnodes = solverF.get_coordinates_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)

In [4]:
# solid solver
jobname = 'solid/solidN'
output = 'asc'
prob = Problem(jobname=jobname, output=output)
msolver = prob.get_solver()
msolver.initailize()
# since we know there is one step
# we stop b4 it
msolver.solve(stopb4step=1)
nlgeom = msolver.get_nlgeomsolver()
nlgeom.initialize()

# initialize our interface set
iset = nlgeom.get_set(
    set_name='Interface',
    set_type='sfelem',
    is_surface=True
)
nset = nlgeom.get_set(
    set_name='Inodes',
    set_type='node',
    is_surface=True
)

# retrieve geometry data
snodes = nset.coordinates()
scents = iset.extract_face_centers()

# create cht adapter
solverS = CHTAdapter(nlgeom=nlgeom)
solverS.add_interface(set_acc=nset, is_settable=False, itype=CHTAdapter.TEMP)
solverS.add_interface(set_acc=iset, is_settable=True, itype=CHTAdapter.DFLUX)

# now we are ready

In [5]:
Tf = 'Tf'
Ts = 'Ts'
Ff = 'Ff'
Fs = 'Fs'
Ff2Fs = {Ff:Fs}
Ts2Tf = {Ts:Tf}
f2s_mapper, s2f_mapper = generate_mapper(fnodes, [scents, snodes], Ff2Fs, Ts2Tf,r1=0.05,r2=0.3)

In [6]:
# coupling data and setups
fluxF = InterfaceData(size=fnodes.shape[0], value=0.0)
fluxS = InterfaceData(size=scents.shape[0], value=0.0)
tempF = InterfaceData(size=fnodes.shape[0], value=350.0)
tempS = InterfaceData(size=snodes.shape[0], value=300.0)
thetaT = InterfaceData(size=fnodes.shape[0], value=0.0)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=tol)
conv_solution = AbsCovergenceMonitor(tol=1e-4)

# maximum pc steps allowed
max_pc_steps = 200

In [7]:
# coupling timing information, use fixed time step for now
# Tfinal = 1.0
# N = 100
# dt = Tfinal/N
dt = 0.05
t = 0.0
Nmax = 100
avg_pc_iterations = 0.0
flog = open('FDSN_ori.log', mode='w')
flog.write('Fluid Dirichlet with solid Neumann setting\n')
flog.close()
flog = open('FDSN_ori.log', mode='a')

In [8]:
# begin to couple
for step in range(Nmax):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup_state()
    
    # mapping, T:s->f
    s2f_mapper.put(Ts, tempS.curr)
    s2f_mapper.apply(Ts, Tf)
    tempF.curr[:] = s2f_mapper.get(Tf)
    
    # update fluid interface temperature
    solverF.put_temperature_interface(
        grid=interface_grid,
        side=interface_side,
        axis=interface_axis,
        data=tempF.curr
    )
    
    while True:
        # back up previous interface value
        tempF.backup()
        
        # advance fluid
        solverF.advance(t=t, dt=dt)
        # retrieve fluid interface flux
        fluxF.curr[:] = solverF.get_heat_flux_interface(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # mapping, F:f->s
        f2s_mapper.put(Ff, fluxF.curr)
        f2s_mapper.apply(Ff, Fs)
        fluxS.curr[:] = f2s_mapper.get(Fs)
        # update the distributed heat fluxes on solid interface
        solverS['Interface', SET].set_dfluxes(fluxS.curr)
        
        # advance solid
        solverS.adjust_timesize(dt)
        solverS.increment()
        # retrieve solid interface temperature
        tempS.curr[:] = solverS['Inodes', GET].get_temperatures()
        
        # mapping, T:s->f
        s2f_mapper.put(Ts, tempS.curr)
        s2f_mapper.apply(Ts, Tf)
        tempF.curr[:] = s2f_mapper.get(Tf)
        
        # update residual
        tempF.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(tempF)
        if is_conv or pc_counts >= max_pc_steps:
            solverS.finish_increment()
            break
        else:
            # if not converge, then underrelaxation and update to fluid then restore
            if use_relax:
                under_relax.determine_omega(tempF, step, pc_counts)
                under_relax.update_solution(tempF)
            # update the underelaxed interface temperature
            # make sure that restoring the fluid solver will
            # not affect the newly updated values
            solverF.put_temperature_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=tempF.curr
            )
            solverF.restore_previous_solutions()
            solverS.restore_state()
            solverS.finish_increment()
            pc_counts = pc_counts + 1

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    avg_pc_iterations += pc_counts
    # fluid output
    if step % 50 == 0 or step == Nmax-1:
        fluid_ofile.write(solverF, t)
flog.close()

nlgeom.finalize()
msolver.solve(skipsolve=True)
msolver.finalize()

step=0, pc_iterations=5.
step=1, pc_iterations=6.
step=2, pc_iterations=5.
step=3, pc_iterations=4.
step=4, pc_iterations=4.
step=5, pc_iterations=4.
step=6, pc_iterations=4.
step=7, pc_iterations=4.
step=8, pc_iterations=3.
step=9, pc_iterations=3.
step=10, pc_iterations=3.
step=11, pc_iterations=2.
step=12, pc_iterations=2.
step=13, pc_iterations=3.
step=14, pc_iterations=3.
step=15, pc_iterations=2.
step=16, pc_iterations=2.
step=17, pc_iterations=2.
step=18, pc_iterations=2.
step=19, pc_iterations=2.
step=20, pc_iterations=2.
step=21, pc_iterations=2.
step=22, pc_iterations=2.
step=23, pc_iterations=2.
step=24, pc_iterations=2.
step=25, pc_iterations=0.
step=26, pc_iterations=0.
step=27, pc_iterations=0.
step=28, pc_iterations=0.
step=29, pc_iterations=0.
step=30, pc_iterations=0.
step=31, pc_iterations=0.
step=32, pc_iterations=0.
step=33, pc_iterations=0.
step=34, pc_iterations=0.
step=35, pc_iterations=0.
step=36, pc_iterations=0.
step=37, pc_iterations=0.
step=38, pc_iterations

In [10]:
theta=(solverF.get_temperature_interface(2,0,1)[0:100]-300)/50
print(theta)
sio.savemat(
    'FDSN_ori.mat',
    {
        'theta': theta
    }
)

[ 0.2399725   0.23612901  0.23235641  0.22865425  0.22502178  0.2214581
  0.21796213  0.21453265  0.21116834  0.20786773  0.20462924  0.2014512
  0.19833182  0.19526928  0.19226172  0.18930725  0.18640399  0.18355003
  0.18074346  0.17798237  0.17526483  0.17258894  0.16995284  0.16735468
  0.16479264  0.16226486  0.15976932  0.15730365  0.15486498  0.15244959
  0.15005268  0.14766837  0.14529024  0.14291259  0.14053226  0.13814989
  0.13576976  0.1333982   0.13104212  0.1287083   0.12640355  0.12413493
  0.12190963  0.11973465  0.11761649  0.11556106  0.11357343  0.11165776
  0.10981732  0.10805483  0.10637248  0.10477189  0.10325401  0.10181887
  0.10046556  0.09919246  0.09799762  0.0968789   0.09583402  0.09486053
  0.0939556   0.09311606  0.09233833  0.09161856  0.09095268  0.09033651
  0.08976602  0.08923755  0.08874794  0.08829452  0.087875    0.08748729
  0.08712945  0.08679968  0.08649628  0.08621773  0.08596261  0.08572967
  0.0855178   0.08532602  0.08515347  0.08499941  0.0